## Main Framework
- Group 9
- Team Member: Ge Zeng, Xiaoying(Shawn) Zou, Xinyue Yu, Yufeng Xie, Yu Cao, Zirui Zhou

In [2]:
# Main Framework
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
# dashboards
# import dash_core_components as dcc

import pandas as pd
import numpy as np
import dash 
import jupyter_dash
from jupyter_dash import JupyterDash # for runnin the Dashboard in jupyter notebook
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc # for Dashboard themes 

#import dash_html_components as html
from dash import dash_table
#import dash_table
from datetime import date
from collections import OrderedDict
from dash.exceptions import PreventUpdate

# other
import matplotlib.cbook as cbook
from sklearn.linear_model import LogisticRegression
from IPython.display import display as d
from IPython.display import Audio
############################################################


# # dashboards
# import dash 
# from jupyter_dash import JupyterDash
# import dash_core_components as dcc
# import dash_html_components as html
# from dash.dependencies import Output, Input
# import dash_bootstrap_components as dbc
# from datetime import date



#################################################################################################################################################################################


## Tab1 Data Preparasion

# download icon
FONT_AWESOME = ("https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css")

# color
# grey: 'rgb(217,217,217)'
colorlist=['rgb(217,241,246)',
'rgb(179,227,237)',
'rgb(140,213,228)',
'rgb(64,185,210)',
'rgb(39,145,166)',
'rgb(26,96,111)']
#Color-dummy pie chart color
pie_color={'Gender':['rgb(217,241,246)','rgb(64,185,210)'],'Customer Type':['rgb(217,241,246)','rgb(64,185,210)'],\
              'Type of Travel':['rgb(217,241,246)','rgb(64,185,210)'],\
              'Class':['rgb(217,241,246)','rgb(64,185,210)','rgb(26,96,111)']}

# app- summary part
app = JupyterDash(__name__,external_stylesheets = [dbc.themes.BOOTSTRAP,FONT_AWESOME])
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/XinyueOliviaYu/plotly_dashboard/main/airline%20passenger%20satisfaction.csv')
df=df.assign(satisfaction_dummy=[1 if i=='satisfied' else 0 for i in df['satisfaction']])
df['Customer Type']=[i if i=='Loyal Customer' else 'Disloyal Customer' for i in df['Customer Type']]

df['Delay']=[0 if i==0 else 1 for i in df['Departure Delay in Minutes']]
df['Delay_time']=['No Delay' if i==0 
                       else '1-6' if i>0 and i<=6 
                       else '6-16' if i<=16 and i>6 
                       else '16-40' if i<=40 and i>16 
                       else '>40' for i in df['Departure Delay in Minutes']]
df['Flight_distance']=['Short' if i<=483 and i>0 
                            else 'Mid' if i<=1513 
                            else 'Long' for i in df['Flight Distance']]
df['Flight_Distance']=['0-483' if i<=483 and i>0 
                            else '483-1513' if i<=1513 
                            else '> 1513' for i in df['Flight Distance']]
df['satisfaction_dummy']=[1 if i=='satisfied' else 0 for i in df['satisfaction']]
df['Age_class']=['Under 21' if i<21
                            else '21-30' if i <=30 
                            else '31-40' if i <=40 
                            else '41-50' if i <=50
                            else '51-60' if i <=60
                            else 'Over 60' for i in df['Age']]
df_delay=df.loc[df.Delay==1]
category_delay_time=pd.DataFrame(df.groupby('Delay_time')['satisfaction_dummy'].mean()).reindex(['1-6','6-16','16-40','>40'])

air=pd.read_csv('https://raw.githubusercontent.com/XinyueOliviaYu/plotly_dashboard/main/airline%20passenger%20satisfaction.csv')
air['Type_of_Travel_dummy']=[0 if i=='Personal Travel' else 1 for i in air['Type of Travel']]
air['satisfaction_dummy']=[0 if i=='neutral or dissatisfied' else 1 for i in air['satisfaction']]
air['Class_dummy']=[0 if i=='Eco Plus' else 1 for i in air['Class']]
air['delay_period']=['No Delay' if i==0 else '1-6mins:<Br>25%' if i>0 and i<=6 else '6-16mins:<Br>25%' if i<=16 and i>6 else '16-40mins:<Br>25%' if i<=40 and i>16 else '>40mins:<Br>25%' for i in air['Departure Delay in Minutes']]
air['delay']=['No Delay: 40%' if i==0 else 'Delay: 60%' for i in air['Departure Delay in Minutes']]
df_sun=air.iloc[:,[26,28,29]]
default=df_sun.groupby('delay')['satisfaction_dummy'].mean()
dff=df_sun.loc[df_sun['delay_period']!='No Delay']
change=dff.groupby('delay_period')['satisfaction_dummy'].mean()

df_sun=air.iloc[:,[26,28,29]]
default=df_sun.groupby('delay')['satisfaction_dummy'].mean()

x=default.index
y=np.round(list(default.values),2)


fig_sun = px.sunburst(df_sun, path=['delay', 'delay_period'], values='satisfaction_dummy',color='delay',\
                  color_discrete_map={'Delay: 60%':'rgb(64,185,210)','No Delay: 40%':'rgb(217,217,217)'})
                  
fig_bar = go.Figure(data=[go.Bar(x=x, y=y, text=y, textposition='outside',width=[0.6,0.6])])
fig_bar.update_traces(marker_color=['rgb(64,185,210)','rgb(217,217,217)'],opacity=0.8)

fig_sun.update_layout(title='Proportion of Delay')
fig_bar.update_layout(title='Delay vs. No Delay',
                      yaxis_title='Satisfaction',
                      plot_bgcolor='white')

fig_bar2=go.Figure(data=[go.Bar(x=['1-6mins','6-16mins','16-40mins','>40mins'],y=[0.44,0.42,0.38,0.36],\
                                text=[0.44,0.42,0.38,0.36],textposition='outside',width=[0.6,0.6,0.6,0.6])])
fig_bar2.update_traces(marker_color='rgb(64,185,210)')
fig_bar2.update_layout(title='1-6 mins vs. 6-16 mins vs. 16-40 mins vs. >40 mins',
                      yaxis_title='Satisfaction',
                      plot_bgcolor='white')

# box and bar
fig1 = px.bar(category_delay_time,
             x=category_delay_time.index, y='satisfaction_dummy',
             width=600, height=400)

category_flight_distance=pd.DataFrame(df.groupby('Flight_distance')['satisfaction_dummy'].mean()).reindex(['Short','Mid','Long'])
fig2 = px.bar(category_flight_distance,
             x=category_flight_distance.index, y='satisfaction_dummy',
             width=600, height=400)

category_age_class=pd.DataFrame(df.groupby('Age_class')['satisfaction_dummy'].mean()).reindex(['Under 21','21-30','31-40','41-50','51-60','Over 60'])
fig3 = px.bar(category_age_class,
             x=category_age_class.index, y='satisfaction_dummy',
             width=600, height=400)


df_satisfied=df.loc[df['satisfaction_dummy']==1,:]
df_unsatisfied=df.loc[df['satisfaction_dummy']==0,:]
# box 
fig11 = go.Figure()
fig11.add_trace(go.Box(y=df_satisfied['Departure Delay in Minutes'], name='Satisfied',
                marker_color = 'rgb(64,185,210)'))
fig11.add_trace(go.Box(y=df_unsatisfied['Departure Delay in Minutes'], name = 'Unsatisfied',
                marker_color = 'rgb(217,217,217)'))
fig11.update_layout(title='Departure Delay in Minutes')
fig22 = go.Figure()
fig22.add_trace(go.Box(y=df_satisfied['Flight Distance'], name='Satisfied',
                marker_color = 'rgb(64,185,210)'))
fig22.add_trace(go.Box(y=df_unsatisfied['Flight Distance'], name = 'Unsatisfied',
                marker_color = 'rgb(217,217,217)'))
fig22.update_layout(title='Flight Distance')
fig33 = go.Figure()
fig33.add_trace(go.Box(y=df_satisfied['Age'], name='Satisfied',
                marker_color = 'rgb(64,185,210)'))
fig33.add_trace(go.Box(y=df_unsatisfied['Age'], name = 'Unsatisfied',
                marker_color = 'rgb(217,217,217)'))
fig33.update_layout(title='Age')
# fig11 = px.box(df,
#              y='Departure Delay in Minutes', x='satisfaction_dummy',
#              width=600, height=400)
# fig22 = px.box(df,
#              x='satisfaction_dummy', y='Flight Distance',
#              width=600, height=400)
# fig33 = px.box(df,
#              x='satisfaction_dummy', y='Age',
#              width=600, height=400)
fig1.update_traces(marker_color=colorlist[:-2], marker_line_color=colorlist[:-2],
                marker_line_width=1.5, opacity=1)
fig2.update_traces(marker_color=colorlist[:-3], marker_line_color=colorlist[:-3],
                marker_line_width=1.5, opacity=1)
fig3.update_traces(marker_color=colorlist, marker_line_color=colorlist,
                marker_line_width=1.5, opacity=1)
for i in [fig1,fig11,fig2,fig22,fig3,fig33]:
    i.update_layout(plot_bgcolor='white')


cards = [
    dbc.Card(
        [
            html.H2('129,880', className="card-title"),
            html.P("Total Number of Records", className="card-text"),
        ],
        body=True,
        color="light",
    ),
    dbc.Card(
        [
            html.H2("43.45%", className="card-title"),
            html.P("Total Satisfaction Rate", className="card-text"),
        ],
        body=True,
        color='rgb(64, 185, 210)',
        inverse=True,
    ),
    # dbc.Card(
    #     [
    #         html.H2(f"{dfTrain.shape[0]} / {dfTest.shape[0]}", className="card-title"),
    #         html.P("Train / Test Split", className="card-text"),
    #     ],
    #     body=True,
    #     color="primary",
    #     inverse=True,
    # ),
]
# create variable table
variabledata = OrderedDict(
    [
        ("Variables", ["Satisfaction", "Delay", "Gender", "Customer type", "Type of travel", "Class", "Age", "Flight Distance"]),
        ("Meaning", ["Whether a customer is satisfied with the flight experience. \n Value: Satisfied & Not satisfied", \
                    "Whether the flight is delay.\nValue: delay time, by minutes.",\
                     "Value: Female & male",\
                    "Whether a customer is loyal customer. \n Value: Loyal & Disloyal",\
                    "For customers, the purpose of this flight. \nValue: Business & Personal",\
                     "For customers, the class of this fight. \nValue: Business & Eco",\
                    "Integer Value.",\
                    "The flight distance of each flight. \nValue: distance, by kilometers."]),
    ]
)
variabletable = pd.DataFrame(variabledata)




#################################################################################################################################################################################

## Tab2 Data Preparasion

# Building Heatmap 
df_heat=pd.read_csv('https://raw.githubusercontent.com/XinyueOliviaYu/plotly_dashboard/main/corr.csv',index_col=0)
heatmap = go.Figure()
heatmap.add_trace(go.Heatmap(x = df_heat.columns,y = df_heat.index,z = np.array(df_heat),colorscale='blues'))
heatmap.update_layout( plot_bgcolor="white",title=dict({'text':'Correlation Heatmap of Variables'}),title_xanchor='center',title_x=0.5)


#Building Regression coef bar
data = pd.read_csv('https://raw.githubusercontent.com/XinyueOliviaYu/plotly_dashboard/main/regression_coefs.csv')
data['color']=data['coefs'].apply(lambda x: 'rgb(140,213,228)' if x>=0 else 'rgb((217,217,217))')
data1=data.loc[data['coefs']>=0,:]
data2=data.loc[data['coefs']<0,:]

coefficient = go.Figure()
coefficient.add_trace(go.Bar(
    y=data2['feature'],
    x=data2['coefs'],
    orientation='h',
    name='Coefs>=0',
    marker=dict(
        color='rgb(217,217,217)'
    )
))
coefficient.add_trace(go.Bar(
    y=data1['feature'],
    x=data1['coefs'],
    orientation='h',
    name='Coefs<0',
    marker=dict(
        color='rgb(140,213,228)'
    )
))
coefficient.update_layout( barmode='stack',plot_bgcolor="white",title=dict({'text':'Logistics Regression Coefficients'}),title_xanchor='center',title_x=0.5)

# Radar Chart and Dashboard

df_radar=pd.read_csv('https://raw.githubusercontent.com/XinyueOliviaYu/plotly_dashboard/main/545data_uppercase_2.csv')

satisfaction_all=round(df_radar['satisfaction_01'].sum()/len(df_radar),2)

fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = satisfaction_all,
    #title = {'text': "All Passenger Satisfaction Rate"},
    domain = {'x': [0, 1], 'y': [0, 1]}
))
fig.update_traces(gauge_bar=dict(color='rgb(64,185,210)'))

fig.update_layout(
    title={
        'text': "All Passenger Satisfaction Rate",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font_size':12},
        height=400,width=400)

all_options = {
    'AGE_GROUP': ['51-60', '41-50', '31-40', 'Over 60', '21-30', 'Under 21'],
    'Class': ['Business', 'Eco', 'Eco Plus'],
    'Customer Type':['Loyal Customer', 'Disloyal Customer'],
    'Distance_group':['Mid Distance', 'Long Distance', 'Short Distance'],
    'Gender':['Female', 'Male'],
    'Type of Travel':['Business travel', 'Personal Travel']
}


#################################################################################################################################################################################


## Main Layout

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.Br(),
    # header
    html.H1('Airline Satisfaction', style={'color':'rgb(64,185,210)', 'fontSize': '40px','padding':'0px 0px 0px 20px'}),
    # introduction
    html.Ul([
        html.Li('Introduction: '),
        html.P('''The dashboard aims to help decision makers to gain insights of flight satisfaction and customer attributes. 
        It provides visualization for two facets of analysis.  '''),
        html.P('The first tab demonstrates descriptive analysis including metrics overview, variable explaination and distribution plots. '),
        html.P('The second tab demonstrates correlation heatmap, Logistics regression coefficients (Variables ~ Satisfaction) and interactive Services perception plots for customer subgroups.'),
        html.Li(['Source: ',
                 html.A('Kaggle Link', 
                 href='https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction') 
                 ]),
        html.P(),
        html.Li('Team Member: Ge Zeng, Xiaoying(Shawn) Zou, Xinyue Yu, Yufeng Xie, Yu Cao, Zirui Zhou')
        
    ]),
    # download
    # html.Button("Download CSV", id="btn_csv"),
    
    dbc.Button(id='btn_csv',
            children=[html.I(className="fa fa-download mr-1"), "Download CSV"],
            color="info",
            className="mt-1",
            
        ),       
    dcc.Download(id="download-dataframe-csv"),
    html.Div([html.Br()]),
    dbc.Tabs([
        
        
## Tab 1
        dbc.Tab([          

    #         html.Div(children =[
    

    # # download
    # # html.Button("Download CSV", id="btn_csv"),
    
    # dbc.Button(id='btn_csv',
    #         children=[html.I(className="fa fa-download mr-1"), "Download CSV"],
    #         color="info",
    #         className="mt-1"
    #     ),       
    # dcc.Download(id="download-dataframe-csv")]),     

    html.Br(),
    # indicater
    dbc.Row([dbc.Col(card) for card in cards],style={'color':'rgb(26,96,111)','padding': '0px 0px 0px 20px'}),
    html.Br(),
    
    html.Div(#className="variable-table",
                            children=[dash_table.DataTable(
                                        data=variabletable.to_dict('records'),
                                        columns=[{'id': c, 'name': c} for c in variabletable.columns],
                                        style_as_list_view=True,
                                        style_cell={'padding': '10px','textAlign': 'left',"whiteSpace": "pre-line",'font-family':'sans-serif'},# set whiteSpace to change line
                                        style_header={
                                            'backgroundColor': 'white',
                                            'fontWeight': 'bold'
                                        },
                                        style_table={'padding': '0px 0px 0px 20px'},
                                        # style_cell_conditional=[
                                        #     {
                                        #         'if': {'column_id': c},
                                        #         'textAlign': 'left'
                                        #     } for c in ['Date', 'Region']
                                        # ],
                                        style_data_conditional=[
                                            {
                                                'if': {'row_index': 'odd'},
                                                'backgroundColor': 'rgb(202, 235, 242)',
                                            }
                                        ],
                                    )
                                
                            ],
),
    html.Br(),
    html.Br(),
    # sunburst
    html.H3('Delay & Satisfaction Overview',style={'color':'rgb(26,96,111)', 'fontSize': '25px', 'padding': '20px 20px 20px 20px'}),

    html.Br(),

    html.Div([dbc.Button( id='delay-button',children=[html.I(className="fa fa-download mr-1"), 'Detailed Graph'],color="info", className='me-1')]),
      

    html.Div(dcc.Graph(id='delay-sunburst',figure=fig_sun),style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div(dcc.Graph(id='delay-satisfaction', figure=fig_bar),style={'display': 'inline-block', 'width': '49%'}),
    

    # pie chart + bar chart
    html.H3('Distribution Plot',style={'color':'rgb(26,96,111)', 'fontSize': '25px', 'padding': '20px 20px 20px 20px'}),
    html.Li("Categorical Variables:",style={'color':'rgb(26,96,111)','fontSize': '20px','padding': '20px 20px 20px 20px'}),
    dcc.Dropdown(
        id='dropdown',
        options=[
            {'label': 'Gender', 'value': 'Gender'},
            {'label': 'Class', 'value': 'Class'},
            {'label': 'Customer Type', 'value': 'Customer Type'},
            {'label': 'Type of Travel', 'value': 'Type of Travel'}
        ],
        value='Gender',
        style={'color':'rgb(26,96,111)','padding': '0px 0px 0px 20px'}
    ),
    html.Br(),
    html.Div([
        dcc.Graph(id="piechart")
        ], style={'width': '49%', 'display': 'inline-block'}),
    html.Div([
        dcc.Graph(id="barchart")
        ], style={'width': '49%', 'display': 'inline-block'}),
    
    html.Li("Continuous Variables:",style={'color':'rgb(26,96,111)','fontSize': '20px','padding': '20px 20px 20px 20px'}),
    # box plot
    dcc.Graph(id='11',figure=fig11,
                        style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    dcc.Graph(id='1',figure=fig1,
                        style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    dcc.Graph(id='22',figure=fig22,
                        style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    dcc.Graph(id='2',figure=fig2,
                        style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    dcc.Graph(id='33',figure=fig33,
                        style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    dcc.Graph(id='3',figure=fig3,
                        style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'})
    ],label='Basic Descriptive'),


#################################################################################################################################################################################

## Tab 2
        dbc.Tab([
            html.H1('Correlation',style={'color':'rgb(26,96,111)', 'fontSize': '25px', 'padding': '20px 20px 20px 20px'}),
                html.Div([
                
                #Heatmap Layout  
                html.Div([dcc.Graph(id="heatmap",figure=heatmap,style={'width': '90vh', 'height': '90vh'})],\
                        style={'width': '49%','display':'inline-block'}),  #style={'width': '200vh', 'height': '100vh'}

                # Logistics Coefficient
                html.Div([dcc.Graph(id="coefficient_bar",figure=coefficient, style={'width': '90vh', 'height': '90vh'})],\
                        style={'width': '49%','float':'right','display':'inline-block'}),
                ]),
            
            html.Br(),
            html.H1('Interaction',style={'color':'rgb(26,96,111)', 'fontSize': '25px','padding': '20px 20px 20px 20px'}),
                html.Div([
                # Radar Chart and Dashboard
                

                html.Div([
                html.Div([
                dcc.Dropdown(
                    id='dropdown1',
                    options=[{'label': i, 'value': i} for i in all_options.keys()],
                    value='Gender'
                ),
                dcc.Dropdown(
                    id='dropdown2'
                )
                ], style={'width': '20%', 'display': 'inline-block'}), 

                html.Div([
                dcc.Dropdown(
                    id='dropdown3',
                    options=[{'label': i, 'value': i} for i in all_options.keys()],
                    value='Gender'
                ),
                dcc.Dropdown(
                    id='dropdown4',
                )
                ], style={'width': '20%', 'display': 'inline-block'})],style={'padding': '10px 50px 10px 500px'}),
                
                 html.Div([
                dcc.Graph(id='indicator_all',figure=fig)
                ], style={'width': '33%', 'display': 'inline-block'}),

                html.Div([
                dcc.Graph(id='indicator')
                ], style={'width': '33%', 'display': 'inline-block'}),

            
                html.Div([
                dcc.Graph(id='indicator_radar')
                ], style={'width': '33%', 'display': 'inline-block'})

                ]),
        ],label='Correlation & Interaction')
    ])
])

#################################################################################################################################################################################


## Tab1 Callback Function

# download                        
@app.callback(
            Output("download-dataframe-csv", "data"),
            Input("btn_csv", "n_clicks"),
            prevent_initial_call=True,
        )
def func(n_clicks):
    return dcc.send_data_frame(df.to_csv, "airline passenger satisfaction.csv")

@app.callback(
    Output('piechart', 'figure'),
    Input('dropdown', 'value')
)
def piechart_output(value):
    df1=df[value].value_counts()
    fig = go.Figure()
    fig.add_trace(go.Pie(labels=df1.index, values=df1.values, marker_colors=pie_color[value],\
                        textinfo='label+percent'))
    return fig

@app.callback(
    Output('barchart', 'figure'),
    Input('dropdown', 'value')
)
def barchart_output(value):
    df2=df.groupby(value)['satisfaction_dummy'].value_counts().unstack()
    df2['Satisfaction Rate']=df2[1]/(df2[0]+df2[1])
    fig = go.Figure()
    fig.add_trace(go.Bar(y=df2.index, x=df2['Satisfaction Rate'],orientation='h',marker_color=pie_color[value]))
    fig.update_layout(plot_bgcolor='white')
    return fig

# sunburst
@app.callback(
    dash.dependencies.Output('delay-satisfaction', 'figure'),
    dash.dependencies.Input('delay-button','n_clicks'))
def button_delay(n_clicks):
    if n_clicks is None:
        raise PreventUpdate
    elif n_clicks%2 == 0:
        return fig_bar
    else: 
        return fig_bar2


#################################################################################################################################################################################


## Tab2 Callback Function
@app.callback(
    Output('dropdown2', 'options'),
    Input('dropdown1', 'value'))
def updatedropdown_option(value):
    return [{'label': i, 'value': i} for i in all_options[value]]


@app.callback(
    Output('dropdown2', 'value'),
    Input('dropdown1', 'options'))
def updatedropdown_value(available_options):
    return available_options[0]['value']


@app.callback(
    Output('dropdown4', 'options'),
    Input('dropdown3', 'value'))
def updatedropdown_option(value):
    return [{'label': i, 'value': i} for i in all_options[value]]


@app.callback(
    Output('dropdown4', 'value'),
    Input('dropdown3', 'options'))
def updatedropdown_value(available_options):
    return available_options[0]['value']

# define callback for updating the indicator chart
@app.callback(
    dash.dependencies.Output('indicator', 'figure'),
    [dash.dependencies.Input('dropdown1', 'value'),
     dash.dependencies.Input('dropdown2', 'value'),
     dash.dependencies.Input('dropdown3', 'value'),
     dash.dependencies.Input('dropdown4', 'value')
     ])
def update_indicator(dropdown_1,dropdown_2,dropdown_3,dropdown_4):

    df1=df_radar.loc[(df_radar[dropdown_1]==dropdown_2)&(df_radar[dropdown_3]==dropdown_4)]
    satisfaction=round(df1['satisfaction_01'].sum()/len(df1),2)
    
    fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = satisfaction,
    #title = {'text': dropdown_2+' & '+ dropdown_4 + ' '+"Subgroup Satisfaction Rate"},
    domain = {'x': [0, 1], 'y': [0, 1]}
    ))
    
    fig.update_traces(gauge_bar=dict(color='rgb(64,185,210)'))
    
    fig.update_layout(
    title=dict(
        text=dropdown_2+' & '+ dropdown_4 + ' '+"Subgroup Satisfaction Rate",
        y=0.9,
        x=0.5,
        xanchor='center',
        yanchor= 'top',
        font_size=12),
        height=400,
        width=400)

    return fig

@app.callback(
    dash.dependencies.Output('indicator_radar', 'figure'),
    [dash.dependencies.Input('dropdown1', 'value'),
     dash.dependencies.Input('dropdown2', 'value'),
     dash.dependencies.Input('dropdown3', 'value'),
     dash.dependencies.Input('dropdown4', 'value')
     ])
def update_indicator_radar(dropdown_1,dropdown_2,dropdown_3,dropdown_4):

    df1=df_radar.loc[(df_radar[dropdown_1]==dropdown_2)&(df_radar[dropdown_3]==dropdown_4)]
    categories = ['Wifi','Food & Drinks','Entertainment','Online Boarding', 'Cleanliness']

    r1=round(df1['Inflight wifi service'].sum()/len(df1),2)
    r2=round(df1['Food and drink'].sum()/len(df1),2)
    r3=round(df1['Inflight entertainment'].sum()/len(df1),2)
    r4=round(df1['Online boarding'].sum()/len(df1),2)
    r5=round(df1['Cleanliness'].sum()/len(df1),2)

    r1_all=round(df_radar['Inflight wifi service'].sum()/len(df_radar),2)
    r2_all=round(df_radar['Food and drink'].sum()/len(df_radar),2)
    r3_all=round(df_radar['Inflight entertainment'].sum()/len(df_radar),2)
    r4_all=round(df_radar['Online boarding'].sum()/len(df_radar),2)
    r5_all=round(df_radar['Cleanliness'].sum()/len(df_radar),2)

    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=[r1,r2,r3,r4,r5],
        theta=categories,
        fill='toself',
        name='Subgroup',
        marker_color='rgb(255,214,99)'
    ))
    
    
    fig.add_trace(go.Scatterpolar(
        r=[r1_all,r2_all,r3_all,r4_all,r5_all],
        theta=categories,
        fill='toself',
        name='All Passenger',
        marker_color='rgb(64,185,210)'
    ))

    fig.update_layout(
    polar=dict(
        radialaxis=dict(
        visible=True,
        range=[0, 5]
        )),
    showlegend=False
    )

    fig.update_layout(
    title={
        'text': dropdown_2+' & '+ dropdown_4 + ' '+"<Br>Subgroup and All Passenger Service Rating",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font_size':15},
        height=400,width=400)
    
    return fig


#################################################################################################################################################################################


if __name__ == '__main__':
    app.run_server(debug=True, height= 1000, width = '100%',port=8055)

Dash app running on http://127.0.0.1:8055/


<ipython-input-2-4f881c8d60df>:602: RuntimeWarning:

invalid value encountered in longlong_scalars

<ipython-input-2-4f881c8d60df>:638: RuntimeWarning:

invalid value encountered in longlong_scalars

<ipython-input-2-4f881c8d60df>:639: RuntimeWarning:

invalid value encountered in longlong_scalars

<ipython-input-2-4f881c8d60df>:640: RuntimeWarning:

invalid value encountered in longlong_scalars

<ipython-input-2-4f881c8d60df>:641: RuntimeWarning:

invalid value encountered in longlong_scalars

<ipython-input-2-4f881c8d60df>:642: RuntimeWarning:

invalid value encountered in longlong_scalars

